In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import json
import time
import pandas as pd

In [27]:
# setting the url

url = 'https://www.foodpanda.pk/restaurants/new?lat=24.88230453160822&lng=67.06722308262242&vertical=restaurants'

# setting the web driver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
user_agent = 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))

service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(options=options, service=service)
driver = webdriver.Chrome(executable_path='My Labs/chromedriver.exe')

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """
})

# going to the link in url variable

driver.get(url)
driver.maximize_window()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

# loading all the restaurants on that page

for spin in range(3):
    try:
        spinner = driver.find_element(By.XPATH, value='//div[@class="spinner-container"]')
        ActionChains(driver).move_to_element(spinner).perform()
    except:
        break

# geting all the links of restaurants on that page      

links = wait.until(ec.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.vendor-list>li")))        
url_links = []

for link in links:
    try:
        urls = link.find_element(By.TAG_NAME, "a").get_attribute("href")
        url_links.append(urls)
    except:
        pass

# getting the details of restaurants

names = []
rating = []
rating_count = []

for page in url_links[0:12]:
    driver.get(page)
    try:
        jsonSource = wait.until(ec.presence_of_element_located((By.XPATH, '//script[contains(text(), "@id")]')))
        jsonSource = jsonSource.get_attribute('text')
        jsonSource = json.loads(jsonSource)
        names.append(jsonSource['name'])
        if 'aggregateRating' not in jsonSource:
            rating.append(0)
            rating_count.append(0)
        else:
            rating.append(jsonSource['aggregateRating']['ratingValue'])
            rating_count.append(jsonSource['aggregateRating']['ratingCount'])
    except:
        continue
        
driver.close()

In [28]:
# converting raw data into dataframe 

data = {"name":names, "rating":rating, "rating_count":rating_count}
df = pd.DataFrame(data)

In [29]:
# sorting the data in order

df.sort_values(['rating_count', 'rating'],ascending = [False, False])

,name,rating,rating_count
3,Kababjees Bakers - Sharfabad,4.4,27004
2,KBC - Sharfabad,4.2,8284
1,Kababjees Express - Dhoraji,4.3,8048
4,Hardee's - PECHS,4.3,7648
11,Miraal,4.0,5787
5,Tooso Bahadurabad,4.1,5586
6,ZamZam Bakers & Sweets - Garden,4.2,4744
10,Mad Cheese - PECHS,4.1,3848
8,La Farine - Sharafabad,4.4,1277
9,WALLA,4.2,585


[]